In [1]:
import pandas as pd
train = pd.read_csv("train.csv", header = None)
test = pd.read_csv("test.csv", header = None)

# minor_size is number of classes less than 500 instances
minor_size = 0
grouped = train.groupby(train.columns[-1])
for k,group in grouped:
        if len(group.index) < 500:
            minor_size += 1
        print ("Class label: ",k,end='')
        print("  Number:",len(group.index))
minor_size

Class label:  1  Number: 578
Class label:  2  Number: 579
Class label:  3  Number: 590
Class label:  4  Number: 571
Class label:  5  Number: 577
Class label:  6  Number: 580
Class label:  7  Number: 570
Class label:  8  Number: 580
Class label:  9  Number: 586
Class label:  10  Number: 579
Class label:  11  Number: 190
Class label:  12  Number: 195
Class label:  13  Number: 196
Class label:  14  Number: 190
Class label:  15  Number: 190
Class label:  16  Number: 194
Class label:  17  Number: 193
Class label:  18  Number: 190
Class label:  19  Number: 193
Class label:  20  Number: 195
Class label:  21  Number: 98
Class label:  22  Number: 97
Class label:  23  Number: 97
Class label:  24  Number: 98
Class label:  25  Number: 96
Class label:  26  Number: 98
Class label:  27  Number: 98
Class label:  28  Number: 97
Class label:  29  Number: 97
Class label:  30  Number: 95
Class label:  31  Number: 49
Class label:  32  Number: 48
Class label:  33  Number: 49
Class label:  34  Number: 48
Cla

39

In [2]:
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import cross_validation
# exclude sample ID (1st column)
x_train_valid = train.iloc[:,1:-1].to_numpy()
y_train_valid = train.iloc[:,-1].to_numpy()

#split a validation set for hyper-parameter tuning
split_ratio = 0.2
x_train, x_val, y_train, y_val = train_test_split(x_train_valid, y_train_valid, test_size=split_ratio, random_state=1,stratify = y_train_valid) 

x_test = test.iloc[:,1:].to_numpy()




In [3]:
from imblearn.over_sampling import ADASYN, BorderlineSMOTE, SMOTE

# we only oversample during training
ada = ADASYN(random_state = 42, sampling_strategy = 'minority', n_neighbors=5)
bdsmt = BorderlineSMOTE(random_state = 42)
smt = SMOTE(random_state = 42)

x_ada_train = x_train.copy()
y_ada_train = y_train.copy()
x_bdsmt_train = x_train.copy()
y_bdsmt_train = y_train.copy()
x_smt_train = x_train.copy()
y_smt_train = y_train.copy()

# ada failed to resample all minority class at once, we manually resample minority classes(less than 500 instances)
for i in range(minor_size):
    x_ada_train, y_ada_train = ada.fit_resample(x_ada_train, y_ada_train)
    
    
x_bdsmt_train, y_bdsmt_train = bdsmt.fit_resample(x_bdsmt_train, y_bdsmt_train)
x_smt_train, y_smt_train = smt.fit_resample(x_smt_train, y_smt_train)

print("ada size: ", len(y_ada_train))
print("bdsmt size: ", len(y_bdsmt_train))
print("smt size: ", len(y_smt_train))

ada size:  25076
bdsmt size:  22222
smt size:  23128


In [4]:
from sklearn.decomposition import PCA
from sklearn import preprocessing
import numpy as np


# normalize data, useful when no outlier
x_ada_train_norm = preprocessing.normalize(x_ada_train)
x_bdsmt_train_norm = preprocessing.normalize(x_bdsmt_train)
x_val_norm = preprocessing.normalize(x_val)

# standardize data, useful when data is normally distributed
#scaler = preprocessing.RobustScaler().fit(x_bdsmt_train)
scaler = preprocessing.RobustScaler().fit(x_train)
x_bdsmt_train_std = scaler.transform(x_bdsmt_train)
x_train_std = scaler.transform(x_train)
x_val_std = scaler.transform(x_val)
x_test_std = scaler.transform(x_test)


'''
pca = PCA(n_components=300)
pca = PCA(n_components = 'mle')

x_ada_train_pca =  pca.fit_transform(x_ada_train_norm)
x_bdsmt_train_pca =  pca.fit_transform(x_bdsmt_train_norm)
x_val_pca = pca.fit_transform(x_val_norm)

print(x_ada_train_pca.shape, x_bdsmt_train_pca.shape)
'''

"\npca = PCA(n_components=300)\npca = PCA(n_components = 'mle')\n\nx_ada_train_pca =  pca.fit_transform(x_ada_train_norm)\nx_bdsmt_train_pca =  pca.fit_transform(x_bdsmt_train_norm)\nx_val_pca = pca.fit_transform(x_val_norm)\n\nprint(x_ada_train_pca.shape, x_bdsmt_train_pca.shape)\n"

In [5]:
# feature selection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import GenericUnivariateSelect, f_classif

# selectfrommodel using extra tree
clf = ExtraTreesClassifier(n_estimators=50)
clf.fit(x_bdsmt_train, y_bdsmt_train)
selection_model = SelectFromModel(clf, prefit=True)
x_bdsmt_train_selected = selection_model.transform(x_bdsmt_train)
x_val_selected = selection_model.transform(x_val)
x_test_selected = selection_model.transform(x_test)
x_bdsmt_train_selected.shape

(22222, 365)

In [6]:
# GenericUnivariateSelect using f_classif(ANOVA f-value)
transformer = GenericUnivariateSelect(f_classif, mode='k_best', param=300)
transformer.fit_transform(x_bdsmt_train, y_bdsmt_train)
x_train_tran = transformer.transform(x_train)
x_bdsmt_train_tran = transformer.transform(x_bdsmt_train)
x_val_tran = transformer.transform(x_val)
x_test_tran = transformer.transform(x_test)
x_bdsmt_train_tran.shape

(22222, 300)

In [7]:
'''
attempt on SVM

from sklearn.utils import class_weight
from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np

clf_svm_weight = svm.SVC(class_weight = "balanced")
clf_svm_weight.fit(x_train, y_train)

y_pred_weight = clf_svm_weight.predict(x_val)
accuracy_score(y_val, y_pred_weight)
'''

'\nattempt on SVM\n\nfrom sklearn.utils import class_weight\nfrom sklearn import svm\nfrom sklearn.metrics import accuracy_score\nimport numpy as np\n\nclf_svm_weight = svm.SVC(class_weight = "balanced")\nclf_svm_weight.fit(x_train, y_train)\n\ny_pred_weight = clf_svm_weight.predict(x_val)\naccuracy_score(y_val, y_pred_weight)\n'

In [8]:
'''
clf_svm = svm.SVC()
clf_svm.fit(x_bdsmt_train_std, y_bdsmt_train)

y_pred = clf_svm.predict(x_val_std)
accuracy_score(y_val, y_pred)

y_pred_test = clf_svm.predict(x_test)
id_svm = test.iloc[:,0].to_numpy()
id_svm = [(i-1) for i in id_svm]
df = pd.DataFrame({'Id': id_svm, 'Category': y_pred_test})
df.to_csv("svm_output.csv", index = False)
'''


'\nclf_svm = svm.SVC()\nclf_svm.fit(x_bdsmt_train_std, y_bdsmt_train)\n\ny_pred = clf_svm.predict(x_val_std)\naccuracy_score(y_val, y_pred)\n\ny_pred_test = clf_svm.predict(x_test)\nid_svm = test.iloc[:,0].to_numpy()\nid_svm = [(i-1) for i in id_svm]\ndf = pd.DataFrame({\'Id\': id_svm, \'Category\': y_pred_test})\ndf.to_csv("svm_output.csv", index = False)\n'

In [9]:
'''

import xgboost as xgb
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import accuracy_score

space={
        'n_estimators': hp.quniform('n_estimators', 50, 300, 50),
        'eta': hp.quniform('eta', 0.1,0.8,0.05),
        'max_depth': hp.quniform("max_depth", 3, 12, 1),
        'gamma': hp.uniform ('gamma', 0,10),
        'reg_alpha' : hp.quniform('reg_alpha', 0,100,1),
        'reg_lambda' : hp.quniform('reg_lambda', 0,100,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.2,1),
        'colsample_bynode' : hp.uniform('colsample_bynode', 0.2,1),
        'colsample_bylevel' : hp.uniform('colsample_bylevel', 0.2,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'sampling_method' : hp.choice('sampling_method', ('uniform', 'gradient_based')),
        'seed': 0
    }

def objective(space):
   
    clf=xgb.XGBClassifier(
                  space, tree_method = 'gpu_hist')
    print("space: ", space)
    
    evaluation = [(x_bdsmt_train, y_bdsmt_train), ( x_val, y_val)]
    
    clf.fit(x_bdsmt_train, y_bdsmt_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=15,verbose=False)
    

    pred = clf.predict(x_val)
    #accuracy = accuracy_score(y_val, pred>0.5)
    accuracy = accuracy_score(y_val, pred)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }


trials = Trials()
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 50,
                        trials = trials)
best_hyperparams                 
'''

'\n\nimport xgboost as xgb\nfrom hyperopt import STATUS_OK, Trials, fmin, hp, tpe\nfrom sklearn.metrics import accuracy_score\n\nspace={\n        \'n_estimators\': hp.quniform(\'n_estimators\', 50, 300, 50),\n        \'eta\': hp.quniform(\'eta\', 0.1,0.8,0.05),\n        \'max_depth\': hp.quniform("max_depth", 3, 12, 1),\n        \'gamma\': hp.uniform (\'gamma\', 0,10),\n        \'reg_alpha\' : hp.quniform(\'reg_alpha\', 0,100,1),\n        \'reg_lambda\' : hp.quniform(\'reg_lambda\', 0,100,1),\n        \'colsample_bytree\' : hp.uniform(\'colsample_bytree\', 0.2,1),\n        \'colsample_bynode\' : hp.uniform(\'colsample_bynode\', 0.2,1),\n        \'colsample_bylevel\' : hp.uniform(\'colsample_bylevel\', 0.2,1),\n        \'min_child_weight\' : hp.quniform(\'min_child_weight\', 0, 10, 1),\n        \'sampling_method\' : hp.choice(\'sampling_method\', (\'uniform\', \'gradient_based\')),\n        \'seed\': 0\n    }\n\ndef objective(space):\n   \n    clf=xgb.XGBClassifier(\n                  s

In [10]:
'''
import xgboost as xgb
from sklearn.metrics import accuracy_score
para = {'colsample_bylevel': 0.7506165996395784, 'colsample_bynode': 0.47255523978411235, 'colsample_bytree': 0.6309464903602169, 'eta': 0.75, 'gamma': 0.7838134219171133, 'max_depth': 7.0, 'min_child_weight': 4.0, 'n_estimators': 150.0, 'reg_alpha': 48.0, 'reg_lambda': 44.0, 'sampling_method': 'uniform', 'seed': 0}
clf_xgb=xgb.XGBClassifier(para, tree_method = 'gpu_hist')

clf_xgb.fit(x_train, y_train, verbose = True) 
y_pred_xgb = clf_xgb.predict(x_val)
accuracy_score(y_val, y_pred_xgb)
'''

"\nimport xgboost as xgb\nfrom sklearn.metrics import accuracy_score\npara = {'colsample_bylevel': 0.7506165996395784, 'colsample_bynode': 0.47255523978411235, 'colsample_bytree': 0.6309464903602169, 'eta': 0.75, 'gamma': 0.7838134219171133, 'max_depth': 7.0, 'min_child_weight': 4.0, 'n_estimators': 150.0, 'reg_alpha': 48.0, 'reg_lambda': 44.0, 'sampling_method': 'uniform', 'seed': 0}\nclf_xgb=xgb.XGBClassifier(para, tree_method = 'gpu_hist')\n\nclf_xgb.fit(x_train, y_train, verbose = True) \ny_pred_xgb = clf_xgb.predict(x_val)\naccuracy_score(y_val, y_pred_xgb)\n"

In [11]:
'''
y_pred_xgb = clf_xgb.predict(x_test)
id_svm = test.iloc[:,0].to_numpy()
id_svm = [(i-1) for i in id_svm]
df = pd.DataFrame({'Id': id_svm, 'Category': y_pred_xgb})
df.to_csv("output_xgb.csv", index = False)
'''

'\ny_pred_xgb = clf_xgb.predict(x_test)\nid_svm = test.iloc[:,0].to_numpy()\nid_svm = [(i-1) for i in id_svm]\ndf = pd.DataFrame({\'Id\': id_svm, \'Category\': y_pred_xgb})\ndf.to_csv("output_xgb.csv", index = False)\n'

### LSTM

In [12]:
# Build neural network
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TimeDistributed, Reshape, Dropout, Flatten, LSTM, Dense
from tensorflow.keras.models import Sequential
from keras.layers.convolutional import Conv1D, MaxPooling1D

n_timesteps = 16
n_features = 60
'''
dense_no = 50
inputs_lstm = layers.Input(shape=(960,))
    
reshape_input = layers.Reshape((16,60), input_shape = (960,))(inputs_lstm)
dense1 = layers.Dense(dense_no, activation="relu")
dense2 = layers.Dense(dense_no, activation="relu")

conv1 = Conv1D(filters=32,kernel_size=1,activation='relu', input_shape = (1,n_features))
conv2 = Conv1D(filters=32,kernel_size=1,activation='relu')
dense3 = layers.Dense(dense_no, activation="relu")



shape = reshape_input.get_shape()
concatenate_input = []
stack_input2 = []

print(reshape_input)
for i in range(shape[1]):
    #print(tf.reshape(reshape_input[:,i,:], [-1,60]))
    #concatenate_input.append(dense2(Dropout(0.2)((dense1(tf.reshape(reshape_input[:,i,:], [-1,60]))))))
   
    conv_input = tf.reshape(reshape_input[:,i,:], [-1, 1, 60])
    print(conv_input.shape )
    print(conv1(conv_input).shape)
    conv_output = MaxPooling1D(padding = 'same')((Dropout(0.2)(conv1(conv_input))))
    
    print(conv_output.shape)
    #x = MaxPooling2D((2,2), padding='same')(x)
    conv_output = dense3(Flatten()(MaxPooling1D(padding = 'same')((Dropout(0.2)(conv2(conv_input))))))
    
    concatenate_input.append(conv_output)
    
    #stack_input2.append(tf.reshape(reshape_input[:,i,:], [-1,60]))
print(concatenate_input )
print(stack_input2)
#out = Concatenate()(all_step)
concatenate_input = Concatenate()(concatenate_input)
print(concatenate_input.shape)
concatenate_input = tf.reshape(concatenate_input,[-1,16,dense_no])
x = layers.LSTM(16, dropout=0.3, input_shape = (16,60),return_sequences=True)(concatenate_input)
x = layers.LSTM(16, dropout=0.3, input_shape = (16,60))(x)
outputs = layers.Dense(50, activation="softmax")(x)

model_lstm = keras.Model(inputs = inputs_lstm, outputs = outputs)
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()
'''
model_lstm = Sequential()
model_lstm.add(layers.Reshape((16, 60), input_shape=(960,)))
#model_lstm.add(layers.LSTM(16, dropout=0.15, input_shape = (16,60),return_sequences=True))
model_lstm.add(layers.LSTM(16, dropout=0.15, input_shape = (16,60)))
#model_lstm.add(layers.LSTM(16))
#model_lstm.add(layers.GlobalAveragePooling1D())
model_lstm.add(layers.Dense(50, activation="softmax"))
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 16, 60)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 16)                4928      
_________________________________________________________________
dense (Dense)                (None, 50)                850       
Total params: 5,778
Trainable params: 5,778
Non-trainable params: 0
_________________________________________________________________


In [13]:
model_lstm.fit(x_train_std,y_train, epochs=100, batch_size=64, verbose=True, validation_data=(x_val_std, y_val))    

Epoch 1/100
118/118 [==============================] - 2s 9ms/step - loss: 3.8266 - accuracy: 0.0623 - val_loss: 3.6977 - val_accuracy: 0.1113
Epoch 2/100
118/118 [==============================] - 1s 6ms/step - loss: 3.5250 - accuracy: 0.1214 - val_loss: 3.3560 - val_accuracy: 0.1278
Epoch 3/100
118/118 [==============================] - 1s 6ms/step - loss: 3.2271 - accuracy: 0.1610 - val_loss: 3.1722 - val_accuracy: 0.1656
Epoch 4/100
118/118 [==============================] - 1s 7ms/step - loss: 3.0786 - accuracy: 0.1887 - val_loss: 3.0644 - val_accuracy: 0.1768
Epoch 5/100
118/118 [==============================] - 1s 7ms/step - loss: 2.9764 - accuracy: 0.2017 - val_loss: 2.9840 - val_accuracy: 0.1896
Epoch 6/100
118/118 [==============================] - 1s 8ms/step - loss: 2.9001 - accuracy: 0.2146 - val_loss: 2.9297 - val_accuracy: 0.2023
Epoch 7/100
118/118 [==============================] - 1s 8ms/step - loss: 2.8366 - accuracy: 0.2252 - val_loss: 2.8908 - val_accuracy: 0.2114

Epoch 58/100
118/118 [==============================] - ETA: 0s - loss: 2.0808 - accuracy: 0.42 - 1s 7ms/step - loss: 2.0808 - accuracy: 0.4241 - val_loss: 2.4709 - val_accuracy: 0.3493
Epoch 59/100
118/118 [==============================] - 1s 7ms/step - loss: 2.0810 - accuracy: 0.4225 - val_loss: 2.4833 - val_accuracy: 0.3445
Epoch 60/100
118/118 [==============================] - 1s 8ms/step - loss: 2.0750 - accuracy: 0.4314 - val_loss: 2.4756 - val_accuracy: 0.3472
Epoch 61/100
118/118 [==============================] - 1s 8ms/step - loss: 2.0673 - accuracy: 0.4233 - val_loss: 2.4616 - val_accuracy: 0.3472
Epoch 62/100
118/118 [==============================] - 1s 8ms/step - loss: 2.0604 - accuracy: 0.4300 - val_loss: 2.4785 - val_accuracy: 0.3482
Epoch 63/100
118/118 [==============================] - 1s 7ms/step - loss: 2.0647 - accuracy: 0.4274 - val_loss: 2.4820 - val_accuracy: 0.3504
Epoch 64/100
118/118 [==============================] - 1s 7ms/step - loss: 2.0586 - accuracy:

In [14]:
y_pred_lstm = model_lstm.predict(x_test_std)

y_pred_lstm = [np.argmax(i) for i in y_pred_lstm]

id_lstm = test.iloc[:,0].to_numpy()
id_lstm = [(i-1) for i in id_lstm]
df = pd.DataFrame({'Id': id_lstm, 'Category': y_pred_lstm})
df.to_csv("output_lstm.csv", index = False)


### CNN

In [15]:
from tensorflow.keras import Model, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Reshape, Concatenate
from keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical

n_timesteps = 16
n_features = 60
ff_out = 50

model = Sequential()
model.add(Reshape((n_timesteps,n_features), input_shape = (960,)))
model.add(Conv1D(filters=32,kernel_size=1,activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
#model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
#odel.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(50, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 16, 60)            0         
_________________________________________________________________
module_wrapper (ModuleWrappe (None, 16, 32)            1952      
_________________________________________________________________
dropout (Dropout)            (None, 16, 32)            0         
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 8, 32)             0         
_________________________________________________________________
module_wrapper_2 (ModuleWrap (None, 8, 32)             1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 32)             0         
_________________________________________________________________
module_wrapper_3 (ModuleWrap (None, 4, 32)            

In [16]:
 model.fit(x_train_std,y_train, epochs=100, batch_size=64, verbose=True, validation_data=(x_val_std, y_val))    

Epoch 1/100
118/118 [==============================] - 1s 5ms/step - loss: 3.9702 - accuracy: 0.0889 - val_loss: 3.5514 - val_accuracy: 0.1273
Epoch 2/100
118/118 [==============================] - 0s 3ms/step - loss: 3.4130 - accuracy: 0.1447 - val_loss: 3.3250 - val_accuracy: 0.1597
Epoch 3/100
118/118 [==============================] - 0s 4ms/step - loss: 3.2144 - accuracy: 0.1771 - val_loss: 3.1371 - val_accuracy: 0.1896
Epoch 4/100
118/118 [==============================] - 0s 3ms/step - loss: 3.0254 - accuracy: 0.1961 - val_loss: 2.9916 - val_accuracy: 0.2018
Epoch 5/100
118/118 [==============================] - 0s 4ms/step - loss: 2.8851 - accuracy: 0.2233 - val_loss: 2.8867 - val_accuracy: 0.2242
Epoch 6/100
118/118 [==============================] - 0s 3ms/step - loss: 2.7813 - accuracy: 0.2437 - val_loss: 2.8013 - val_accuracy: 0.2449
Epoch 7/100
118/118 [==============================] - 0s 3ms/step - loss: 2.6969 - accuracy: 0.2687 - val_loss: 2.7463 - val_accuracy: 0.2657

118/118 [==============================] - 0s 3ms/step - loss: 1.7314 - accuracy: 0.5063 - val_loss: 2.2798 - val_accuracy: 0.4031
Epoch 58/100
118/118 [==============================] - 0s 4ms/step - loss: 1.7164 - accuracy: 0.5096 - val_loss: 2.2834 - val_accuracy: 0.4058
Epoch 59/100
118/118 [==============================] - 0s 3ms/step - loss: 1.7125 - accuracy: 0.5093 - val_loss: 2.3001 - val_accuracy: 0.4031
Epoch 60/100
118/118 [==============================] - 0s 3ms/step - loss: 1.7042 - accuracy: 0.5134 - val_loss: 2.3071 - val_accuracy: 0.3994
Epoch 61/100
118/118 [==============================] - 0s 4ms/step - loss: 1.7021 - accuracy: 0.5053 - val_loss: 2.3166 - val_accuracy: 0.4063
Epoch 62/100
118/118 [==============================] - 0s 3ms/step - loss: 1.6970 - accuracy: 0.5124 - val_loss: 2.2941 - val_accuracy: 0.4127
Epoch 63/100
118/118 [==============================] - ETA: 0s - loss: 1.6883 - accuracy: 0.51 - 0s 3ms/step - loss: 1.6996 - accuracy: 0.5158 - val

In [17]:

y_pred_cnn = model.predict(x_test_std)

y_pred_cnn = [np.argmax(i) for i in y_pred_cnn]

id_cnn = test.iloc[:,0].to_numpy()
id_cnn = [(i-1) for i in id_cnn]
df = pd.DataFrame({'Id': id_cnn, 'Category': y_pred_cnn})
df.to_csv("output_cnn.csv", index = False)
